In [7]:
pip install dash pandas numpy plotly socket

ERROR: Could not find a version that satisfies the requirement socket (from versions: none)
ERROR: No matching distribution found for socket
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install dash pandas numpy plotly

Note: you may need to restart the kernel to use updated packages.


In [33]:
import socket
from dash import Dash, html, dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime
import time

def find_free_port():
    """Encuentra un puerto disponible"""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        s.listen(1)
        port = s.getsockname()[1]
    return port

# Inicializar la aplicación Dash con manejo de errores
try:
    app = Dash(__name__)
    
    # Configuración de alarmas por defecto
    DEFAULT_ALARM_THRESHOLD = 25  # m/s
    
    # Layout del dashboard
    app.layout = html.Div([
        # Encabezado
        html.Div([
            html.H1("Monitor de Viento Tiempo Real",  
                    style={'color': '#ffffff'}),  # Cambiado a blanco
            html.H2("DPWorld Lirquen José Gutiérrez", 
                    style={'color': '#2f9fEf'}),  # Cambiado a blanco
            html.Div(id='connection-status',
                    style={'color': '#90EE90', 'marginBottom': '10px'})  # Verde más claro
        ], style={'textAlign': 'center', 'marginBottom': 30}),
        
        # Panel de control
        html.Div([
            html.Div([
                html.H3("Configuración de Alarmas", style={'color': '#ffffff'}),
                html.Label("Umbral de velocidad (m/s):", style={'color': '#ffffff'}),
                dcc.Input(
                    id='alarm-threshold',
                    type='number',
                    value=DEFAULT_ALARM_THRESHOLD,
                    style={'marginLeft': 10}
                ),
            ], style={'width': '50%', 'display': 'inline-block'}),
            
            html.Div([
                html.H3("Estado del Sistema", style={'color': '#ffffff'}),
                html.Div(id='alarm-status', 
                        style={'padding': '10px', 'borderRadius': '5px'})
            ], style={'width': '50%', 'display': 'inline-block'})
        ], style={'padding': 20, 'backgroundColor': '#1e3d59', 'borderRadius': '10px'}),  # Panel azul más oscuro
        
        # Gráficos principales
        html.Div([
            html.Div([
                dcc.Graph(id='wind-rose')
            ], style={'width': '50%', 'display': 'inline-block'}),
            
            html.Div([
                dcc.Graph(id='wind-speed-gauge')
            ], style={'width': '50%', 'display': 'inline-block'})
        ]),
        
        # Historial
        html.Div([
            dcc.Graph(id='wind-history')
        ]),
        
        # Almacenamiento de datos
        dcc.Store(id='wind-data-store'),
        
        # Intervalo de actualización
        dcc.Interval(
            id='interval-component',
            interval=5*1000,  # 5 segundos
            n_intervals=0
        )
    ], style={'backgroundColor': '#2a4d69', 'padding': '20px', 'minHeight': '100vh'})  # Fondo azul principal
    
    # Historial de datos
    wind_history = []
    
    def get_real_time_data():
        """Simula datos en tiempo real - Reemplazar con datos reales"""
        return {
            'speed': np.random.weibull(2) * 10,
            'direction': np.random.uniform(0, 360),
            'timestamp': datetime.now().strftime('%H:%M:%S')
        }
    
    @app.callback(
        [Output('wind-rose', 'figure'),
         Output('wind-speed-gauge', 'figure'),
         Output('wind-history', 'figure'),
         Output('alarm-status', 'children'),
         Output('alarm-status', 'style'),
         Output('connection-status', 'children')],
        [Input('interval-component', 'n_intervals'),
         Input('alarm-threshold', 'value')]
    )
    def update_graphs(n, alarm_threshold):
        try:
            # Obtener nuevos datos
            data = get_real_time_data()
            
            # Actualizar historial
            wind_history.append(data)
            if len(wind_history) > 50:  # Mantener solo últimos 50 registros
                wind_history.pop(0)
            
            # Rosa de los vientos
            wind_rose = go.Figure(go.Barpolar(
                r=[data['speed']],
                theta=[data['direction']],
                width=[10],
                name='Velocidad actual',
                marker_color=['#ffc13b']  # Color de acento amarillo/dorado
            ))
            wind_rose.update_layout(
                title={'text': 'Dirección del Viento', 'font': {'color': '#ffffff'}},
                paper_bgcolor='#1e3d59',  # Fondo azul oscuro
                plot_bgcolor='#1e3d59',   # Fondo azul oscuro
                polar=dict(
                    angularaxis=dict(
                        direction="clockwise",
                        rotation=90,
                        ticktext=['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW'],
                        tickvals=[0, 45, 90, 135, 180, 225, 270, 315],
                        tickfont={'color': '#ffc13b'}
                    ),
                    radialaxis=dict(
                        tickfont={'color': '#ffc13b'}
                    )
                )
            )
            
            # Medidor de velocidad
            gauge = go.Figure(go.Indicator(
                mode="gauge+number",
                value=data['speed'],
                title={'text': "Velocidad del Viento (m/s)", 'font': {'color': '#ffffff'}},
                number={'font': {'color': '#ffffff'}},
                gauge={
                    'axis': {'range': [None, 50], 'tickfont': {'color': '#ffffff'}},
                    'bar': {'color': '#ffc13b'},
                    'threshold': {
                        'line': {'color': "red", 'width': 4},
                        'thickness': 0.75,
                        'value': alarm_threshold
                    }
                }
            ))
            gauge.update_layout(
                paper_bgcolor='#1e3d59',
                plot_bgcolor='#1e3d59'
            )
            
            # Historial
            history = go.Figure(go.Scatter(
                x=[d['timestamp'] for d in wind_history],
                y=[d['speed'] for d in wind_history],
                mode='lines+markers',
                line={'color': '#ffc13b'}
            ))
            history.update_layout(
                title={'text': 'Historial de Velocidad', 'font': {'color': '#ffffff'}},
                paper_bgcolor='#1e3d59',
                plot_bgcolor='#1e3d59',
                xaxis={'title': 'Tiempo', 'color': '#ffffff', 'gridcolor': 'rgba(255,255,255,0.1)'},
                yaxis={'title': 'Velocidad (m/s)', 'color': '#ffffff', 'gridcolor': 'rgba(255,255,255,0.1)'}
            )
            
            # Estado de alarma
            alarm_style = {
                'padding': '10px',
                'borderRadius': '5px',
                'backgroundColor': 'green',
                'color': 'white'
            }
            alarm_text = "Estado: Normal"
            
            if data['speed'] > alarm_threshold:
                alarm_style['backgroundColor'] = 'red'
                alarm_text = "¡ALARMA! Velocidad excesiva"
            
            connection_status = "Conexión: Activa"
            
            return wind_rose, gauge, history, alarm_text, alarm_style, connection_status
            
        except Exception as e:
            return [go.Figure()] * 3, "Error en el sistema", {'backgroundColor': 'yellow'}, f"Error de conexión: {str(e)}"
    
    if __name__ == '__main__':
        # Encontrar un puerto libre
        port = find_free_port()
        print(f"Iniciando servidor en el puerto {port}")
        app.run_server(debug=True, host='0.0.0.0', port=port)
        
except Exception as e:
    print(f"Error al iniciar la aplicación: {str(e)}")

Iniciando servidor en el puerto 51137
